In [1]:
import os
os.environ["PYTHON_EXEC"] = "/Users/jessieyu/.pyenv/versions/qka-demo/bin/python3"
os.environ["NTC_DOC_FILE"] = "runtime/program_doc.json"
os.environ["NTC_PROGRAM_FILE"] = "runtime/qka_runtime_outer.py"

# List available quantum programs

In [2]:
from qiskit import IBMQ

provider = IBMQ.load_account()
provider.runtime.programs()

Quantum-Kernel-Alignment:
  Description: Quantum kernel alignment algorithm that learns, on a given dataset, a quantum kernel maximizing the SVM classification margin.
  Parameters:
    - feature_map:
      Description: An instance of FeatureMapACME in dictionary format used to map classical data into a quantum state space.
      Type: dict
      Required: True
    - data:
      Description: NxD array of training data, where N is the number of samples and D is the feature dimension.
      Type: numpy.ndarray
      Required: True
    - labels:
      Description: Nx1 array of +/-1 labels of the N training samples.
      Type: numpy.ndarray
      Required: True
    - initial_kernel_parameters:
      Description: Initial parameters of the quantum kernel. If not specified, an array of randomly generated numbers is used.
      Type: numpy.ndarray
      Required: False
    - maxiters:
      Description: Number of SPSA optimization steps. Default is 10.
      Type: int
      Required: False
  

# Invoke Quantum Kernel Alignment program

In [3]:
import numpy as np
from qiskit import Aer
from qka.featuremaps import FeatureMapACME
from oct2py import octave
octave.addpath('/Users/jen/Q/code/quantum_kernel_data/')

num_features=10  # number of features in the input data
num_train=10     # number of training samples per class
num_test=7       # number of test samples per class
C=1              # SVM soft-margin penalty
maxiters=21      # number of SPSA iterations

entangler_map=[[0,1],[1,2],[2,3],[3,4],[4,5],[5,6],[6,7],[7,8],[8,9]]

# Generate the data:
state=42 # setting the state for the random number generator
data_plus, data_minus = octave.generate_data(num_train+num_test, state, nout=2)

x_train = np.concatenate((data_plus.T[:num_train], data_minus.T[:num_train]))
y_train = np.concatenate((-1*np.ones(num_train), np.ones(num_train)))

x_test = np.concatenate((data_plus.T[num_train:], data_minus.T[num_train:]))
y_test = np.concatenate((-1*np.ones(num_test), np.ones(num_test)))

# Define the feature map and its initial parameters:
initial_kernel_parameters = [0.1] # np.pi/2 should yield 100% test accuracy
fm = FeatureMapACME(feature_dimension=num_features, entangler_map=entangler_map)

backend = provider.backend.ibmq_qasm_simulator

In [4]:
def interim_result_callback(interim_result):
    print(f"interim result: {interim_result}\n")

In [5]:
# Call QKA runtime program.
runtime_params = {
    'feature_map': fm.to_dict(),
    'data': x_train,
    'labels': y_train,
    'initial_kernel_parameters': initial_kernel_parameters,
    'maxiters': maxiters,
    'C': C
}
options = {'backend_name': backend.name()}
result = provider.runtime.run(program_name="Quantum-Kernel-Alignment",
                              options=options,
                              params=runtime_params,
                              callback=interim_result_callback,
                             ).result()


interim result: {'cost': 10.585411410548094, 'kernel_parameters': array([0.28922397])}

interim result: {'cost': 9.482754041366878, 'kernel_parameters': array([0.55144409])}

interim result: {'cost': 7.170261123846278, 'kernel_parameters': array([0.7874011])}

interim result: {'cost': 5.2901342120561665, 'kernel_parameters': array([0.94536816])}

interim result: {'cost': 4.278665316108828, 'kernel_parameters': array([1.05452627])}

interim result: {'cost': 3.7185993231125707, 'kernel_parameters': array([1.1272473])}

interim result: {'cost': 3.43303394550185, 'kernel_parameters': array([1.18390779])}

interim result: {'cost': 3.2555699898088952, 'kernel_parameters': array([1.2251351])}

interim result: {'cost': 3.1401744410551706, 'kernel_parameters': array([1.25626871])}

interim result: {'cost': 3.0781005931677026, 'kernel_parameters': array([1.28121874])}

interim result: {'cost': 3.0493159681810447, 'kernel_parameters': array([1.29932717])}

interim result: {'cost': 2.9983758632915

In [6]:
print("final result:")
print(f"aligned_kernel_parameters: {result['aligned_kernel_parameters']}")
print(f"aligned_kernel_matrix: {result['aligned_kernel_matrix']}")

final result:
aligned_kernel_parameters: [1.39000383]
aligned_kernel_matrix: [[1.         0.66223145 0.65759277 0.62219238 0.70458984 0.57116699
  0.69287109 0.64807129 0.6237793  0.61425781 0.18359375 0.11657715
  0.19873047 0.28173828 0.20043945 0.21569824 0.22705078 0.26098633
  0.29650879 0.21252441]
 [0.66223145 1.         0.61047363 0.57861328 0.60180664 0.53442383
  0.60412598 0.62182617 0.58227539 0.56481934 0.14306641 0.10534668
  0.24035645 0.28308105 0.22009277 0.25805664 0.1998291  0.26928711
  0.2244873  0.27746582]
 [0.65759277 0.61047363 1.         0.79003906 0.65710449 0.62084961
  0.73828125 0.61535645 0.57165527 0.7010498  0.28601074 0.16650391
  0.25537109 0.32495117 0.25695801 0.35900879 0.34460449 0.37890625
  0.32958984 0.2947998 ]
 [0.62219238 0.57861328 0.79003906 1.         0.61010742 0.78100586
  0.66906738 0.68835449 0.65222168 0.80981445 0.25024414 0.17004395
  0.2590332  0.32775879 0.27355957 0.32189941 0.29980469 0.33190918
  0.30895996 0.30249023]
 [0.704

# Test the aligned kernel

In [7]:
os.environ["NTC_PROGRAM_FILE"] = "runtime/circuit_runner.py"

In [8]:
from qka.kernel_matrix import KernelMatrix
from sklearn.svm import SVC
from sklearn import metrics

# Test the aligned kernel on test data:

kernel_aligned = result['aligned_kernel_matrix']
model = SVC(C=C, kernel='precomputed')
model.fit(X=kernel_aligned, y=y_train)

km = KernelMatrix(feature_map=fm, backend=backend)
kernel_test = km.construct_kernel_matrix(x1_vec=x_test, x2_vec=x_train, parameters=result['aligned_kernel_parameters'])
labels_test = model.predict(X=kernel_test)
accuracy_test = metrics.balanced_accuracy_score(y_true=y_test, y_pred=labels_test)
print(f"accuracy test: {accuracy_test}")

/Users/jen/opt/anaconda3/envs/qiskit0.24.0python3.7/lib/python3.7/site-packages/qiskit/providers/ibmq/ibmqbackend.py:803: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  noise_model=noise_model, **run_config)


accuracy test: 1.0
